In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 199 kB 51.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=976f75ea7b4d2bcce7ebf1302256d9ab9b7d659091a1d20ae5d884a219d3cac0
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("asg").getOrCreate()

In [5]:
import pandas as pd

In [6]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet

--2022-12-13 12:58:15--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.225.141.158, 13.225.141.98, 13.225.141.105, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.225.141.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21777258 (21M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2021-02.parquet’

yellow_tripdata_202 100%[===================>]  20.77M  27.5MB/s    in 0.8s    

2022-12-13 12:58:16 (27.5 MB/s) - ‘yellow_tripdata_2021-02.parquet’ saved [21777258/21777258]



In [63]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-02.parquet

--2022-12-13 14:16:42--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.39.97, 52.85.39.65, 52.85.39.117, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.39.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10645466 (10M) [binary/octet-stream]
Saving to: ‘fhv_tripdata_2021-02.parquet’

fhv_tripdata_2021-0 100%[===================>]  10.15M  45.1MB/s    in 0.2s    

2022-12-13 14:16:43 (45.1 MB/s) - ‘fhv_tripdata_2021-02.parquet’ saved [10645466/10645466]



In [7]:
df = spark.read \
     .option("header", "true") \
     .parquet("/content/yellow_tripdata_2021-02.parquet")

In [64]:
df2 = spark.read \
     .option("header", "true") \
     .parquet("fhv_tripdata_2021-02.parquet")

In [10]:
df.createOrReplaceTempView('taxitrips')

In [9]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-02-01 00:40:47|  2021-02-01 00:48:28|            1.0|          2.3|       1.0|                 N|         141|         226|           2|        8.5|  3.0|    0.5|       0.

In [123]:
df2.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00013|2021-02-01 00:01:00|2021-02-01 01:33:00|        null|        null|   null|                B00014|
|     B00021         |2021-02-01 00:55:40|2021-02-01 01:06:20|       173.0|        82.0|   null|       B00021         |
|     B00021         |2021-02-01 00:14:03|2021-02-01 00:28:37|       173.0|        56.0|   null|       B00021         |
|     B00021         |2021-02-01 00:27:48|2021-02-01 00:35:45|        82.0|       129.0|   null|       B00021         |
|              B00037|2021-02-01 00:12:50|2021-02-01 00:26:38|        null|       225.0|   null|                B00037|
|              B00037|2021-02-01 00:00:3

In [65]:
df2.createOrReplaceTempView('fhv')

### Question 1. How many taxi trips were there on February 15?

In [42]:
fifteentrips=spark.sql('SELECT to_date(tpep_pickup_datetime) AS date, COUNT(tpep_pickup_datetime) AS count_trips FROM taxitrips WHERE dayofmonth(tpep_pickup_datetime) = 15 GROUP BY date')
fifteentrips.show()

+----------+-----------+
|      date|count_trips|
+----------+-----------+
|2021-02-15|      40322|
+----------+-----------+



###Question 2. Find the longest trip for each day?

In [62]:
triptime = spark.sql('SELECT day_of_month, MAX(trip_time_in_sec) AS max_trip_time_in_sec FROM (SELECT dayofmonth(tpep_pickup_datetime) AS day_of_month, (hour(tpep_pickup_datetime)*3600) + (minute(tpep_pickup_datetime)*60) + (second(tpep_pickup_datetime)) AS trip_time_in_sec FROM taxitrips) GROUP BY day_of_month ORDER BY day_of_month')
triptime.show()

+------------+--------------------+
|day_of_month|max_trip_time_in_sec|
+------------+--------------------+
|           1|               86377|
|           2|               86376|
|           3|               86395|
|           4|               86397|
|           5|               86396|
|           6|               86399|
|           7|               86398|
|           8|               86391|
|           9|               86397|
|          10|               86399|
|          11|               86391|
|          12|               86399|
|          13|               86396|
|          14|               86396|
|          15|               86398|
|          16|               86387|
|          17|               86388|
|          18|               86397|
|          19|               86399|
|          20|               86393|
+------------+--------------------+
only showing top 20 rows



###Question 3. Find Top 5 Most frequent `dispatching_base_num`?

In [74]:
dispatching = spark.sql('SELECT dispatching_base_num, COUNT(dispatching_base_num) AS count_dispatching_base_num FROM fhv GROUP BY dispatching_base_num ORDER BY count_dispatching_base_num DESC LIMIT 5')
dispatching.show()

+--------------------+--------------------------+
|dispatching_base_num|count_dispatching_base_num|
+--------------------+--------------------------+
|              B00856|                     35077|
|              B01312|                     33089|
|              B01145|                     31114|
|              B02794|                     30397|
|              B03016|                     29794|
+--------------------+--------------------------+



### Question 4. Find Top 5 Most common location pairs (PUlocationID and DOlocationID)

In [137]:
location = spark.sql('SELECT PUlocationID, DOlocationID FROM(SELECT PUlocationID, DOlocationID, COUNT(PUlocationID) as count_PU, COUNT(PUlocationID) AS count_DO FROM fhv GROUP BY PUlocationID, DOlocationID ORDER BY count_PU DESC, count_DO DESC) LIMIT 5')
location.show()

+------------+------------+
|PUlocationID|DOlocationID|
+------------+------------+
|       123.0|        null|
|       206.0|       206.0|
|       221.0|       206.0|
|       129.0|       129.0|
|         7.0|         7.0|
+------------+------------+

